In [3]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

import os

In [3]:
#getting a list of countres they work in to loop over.

responsa = requests.get('https://www.pedaids.org/impact/where-we-work/')
soup=BeautifulSoup(responsa.text,'html.parser')


countries_link=[]

countries=soup.find_all('a',attrs={'class':'c-country-list__link'})

for country in countries:
    countries_link.append(country.get('href'))


print(countries_link)

['https://www.pedaids.org/country/cameroon/', 'https://www.pedaids.org/country/cote-divoire/', 'https://www.pedaids.org/country/democratic-republic-congo/', 'https://www.pedaids.org/country/eswatini/', 'https://www.pedaids.org/country/india/', 'https://www.pedaids.org/country/kenya/', 'https://www.pedaids.org/country/lesotho/', 'https://www.pedaids.org/country/malawi/', 'https://www.pedaids.org/country/mozambique/', 'https://www.pedaids.org/country/nigeria/', 'https://www.pedaids.org/country/rwanda/', 'https://www.pedaids.org/country/tanzania/', 'https://www.pedaids.org/country/uganda/', 'https://www.pedaids.org/country/zambia/', 'https://www.pedaids.org/country/zimbabwe/']


In [4]:
#get executive summary of each coutry. 

dict_summ_con={}

def add_if_key_not_exist(dict_obj, key, value):
    """ Add new key-value pair to dictionary only if
    key does not exist in dictionary. """
    if key not in dict_obj:
        dict_summ_con.update({key: value})


for country in countries_link:
    responsa = requests.get(country)
    soup=BeautifulSoup(responsa.text,'html.parser')
        
    try:
        country_name=((re.split('-',soup.title.text)[0]).strip())
        
        summary_paragraphs=soup.find_all('div', attrs={'class':"c-section__content"})
        summary_paragraphs=(str(summary_paragraphs))

        func=re.compile(r'(?<=/"></div><p>).+(?=</p>\n<!--)',re.S|re.M)
        x=func.search(summary_paragraphs)
    
        summary=re.split(r'</p>\n<!',x[0])[0]
        summary=re.sub(r'</p>\n<p>','',summary)
        
        add_if_key_not_exist(dict_summ_con, country_name, summary)
        
    except:

        summary_paragraphs=soup.find_all('div', attrs={'class':"c-section__content"})
        summary_paragraphs=(str(summary_paragraphs))
        
        func=re.compile(r'(?<=#414142;">).+(?=</p>\n<!--)',re.S|re.M)
        x=func.search(summary_paragraphs)
        
        summary=re.split(r'</span></p>',x[0])
        
        add_if_key_not_exist(dict_summ_con, country_name, summary)  
        
#Zimbabwe had some different tags around the part we wanted so we had to extract all the text below and delete some tags inside of the block of text. 
    
dict_summ_con['Zimbabwe']='The Elizabeth Glaser Pediatric AIDS Foundation (EGPAF) in Zimbabwe brings both clinical HIV expertise and a focus on strengthening data quality and health systems to address the HIV and AIDS epidemic in-country. Since 2001, EGPAF has been a lead partner to Zimbabwe’s Ministry of Health and Child Care, supporting the national prevention of mother-to-child transmission of HIV (PMTCT) and pediatric HIV care and treatment programs.EGPAF-Zimbabwe has helped the country work toward its goal of near elimination of new pediatric HIV infections by supporting PMTCT and pediatric programs at the facility, district, provincial, and national levels with a primary focus on improving the quality of HIV services in Zimbabwe. In addition to direct health service delivery support, EGPAF-Zimbabwe advocates for policy development aligned with local community needs focusing on PMTCT, the scale-up of early infant HIV diagnosis (EID), and the specific needs of HIV-positive children and adolescents. We also engage in operations research, which aims to increase access and use of the most effective HIV and AIDS services.'
dict_summ_con['Zimbabwe']

'The Elizabeth Glaser Pediatric AIDS Foundation (EGPAF) in Zimbabwe brings both clinical HIV expertise and a focus on strengthening data quality and health systems to address the HIV and AIDS epidemic in-country. Since 2001, EGPAF has been a lead partner to Zimbabwe’s Ministry of Health and Child Care, supporting the national prevention of mother-to-child transmission of HIV (PMTCT) and pediatric HIV care and treatment programs.EGPAF-Zimbabwe has helped the country work toward its goal of near elimination of new pediatric HIV infections by supporting PMTCT and pediatric programs at the facility, district, provincial, and national levels with a primary focus on improving the quality of HIV services in Zimbabwe. In addition to direct health service delivery support, EGPAF-Zimbabwe advocates for policy development aligned with local community needs focusing on PMTCT, the scale-up of early infant HIV diagnosis (EID), and the specific needs of HIV-positive children and adolescents. We also 

In [5]:
# creating a Dataframe object of the summaries. 

df_summaries = pd.DataFrame(dict_summ_con,index=[0]) 
df_summaries

,Cameroon,Côte d’Ivoire,Democratic Republic of Congo,Eswatini,India,Kenya,Lesotho,Malawi,Mozambique,Nigeria,Rwanda,Tanzania,Uganda,Zambia,Zimbabwe
0,"In 2000, the Elizabeth Glaser Pediatric AIDS F...",The Elizabeth Glaser Pediatric AIDS Foundation...,The Elizabeth Glaser Pediatric AIDS Foundation...,The Elizabeth Glaser Pediatric AIDS Foundation...,SAATHII conducts trainings and roll-out of new...,EGPAF began working in Kenya in 2000. The prog...,The Elizabeth Glaser Pediatric AIDS Foundation...,The Elizabeth Glaser Pediatric AIDS Foundation...,"In 2004, the Elizabeth Glaser Pediatric AIDS F...",Through its partnership with the New Horizons ...,The Elizabeth Glaser Pediatric AIDS Foundation...,The Elizabeth Glaser Pediatric AIDS Foundation...,"Since 2000, the Elizabeth Glaser Pediatric AID...","In 2001, the Elizabeth Glaser Pediatric AIDS F...",The Elizabeth Glaser Pediatric AIDS Foundation...


In [6]:
df_summaries=df_summaries.T

In [7]:
df_summaries=df_summaries.reset_index(drop=False)
df_summaries.columns=[['country','summary']]
df_summaries

,country,summary
0,Cameroon,"In 2000, the Elizabeth Glaser Pediatric AIDS F..."
1,Côte d’Ivoire,The Elizabeth Glaser Pediatric AIDS Foundation...
2,Democratic Republic of Congo,The Elizabeth Glaser Pediatric AIDS Foundation...
3,Eswatini,The Elizabeth Glaser Pediatric AIDS Foundation...
4,India,SAATHII conducts trainings and roll-out of new...
5,Kenya,EGPAF began working in Kenya in 2000. The prog...
6,Lesotho,The Elizabeth Glaser Pediatric AIDS Foundation...
7,Malawi,The Elizabeth Glaser Pediatric AIDS Foundation...
8,Mozambique,"In 2004, the Elizabeth Glaser Pediatric AIDS F..."
9,Nigeria,Through its partnership with the New Horizons ...


In [32]:
#made a list of every project accomplishment, prepared a country column and 'key accomplishments column. 

list_of_accomplishment_country=[]
num_acc=[]
country_column_list_country=[]
sections_of_accomplishment_country=[]

for country in countries_link:
    responsa = requests.get(country)
    soup=BeautifulSoup(responsa.text,'html.parser')
    
    
    n=(len(soup.findAll('div' ,attrs={'class':"c-postcard__content"})))
   
    [list_of_accomplishment_country.append(x.text.strip()) for x in soup.findAll('div' ,attrs={'class':"c-postcard__content"})]
    
    country_column_list_country.extend([(soup.find_all('h1',attrs={'class':"c-hero__title"})[0].text)]*n)
    
    sections_of_accomplishment_country.extend([(soup.find_all('h2',attrs={'class':"c-section__title has-flair"})[0].text)] * n)
    



In [5]:
df_accomplishments = pd.DataFrame(list(zip(list_of_accomplishment_country, sections_of_accomplishment_country,country_column_list_country)), 
               columns =['accomplistments', 'sections','country']) 
df_accomplishment


NameError: name 'list_of_accomplishment_country' is not defined

In [10]:
#make a list of each project name, country and description.

country_column_list_country=[]
projects_list=[]
project_desc=[]

for country in countries_link:
    responsa = requests.get(country)
    soup=BeautifulSoup(responsa.text,'html.parser')
    
    
    n=(len(soup.findAll('div' ,attrs={'class':'c-teaser__title'})))
   
    country_column_list_country.extend([(soup.title.text.split(' - ')[0])]*n)
    
    projects=soup.find_all('div', attrs={'class':'c-teaser__title'})
    [projects_list.append(project.text.strip()) for project in projects];
    
    
    projects=soup.find_all('div', attrs={'class':'c-teaser__text'})
    [project_desc.append(project.text.strip()) for project in projects];



In [11]:
df_projects = pd.DataFrame(list(zip(projects_list, project_desc,country_column_list_country)), 
               columns =['project', 'project_description','country']) 
df_projects

,project,project_description,country
0,Delivering Technical Assistance (DELTA) Projec...,"Under Project DELTA, through CDC funding, EGPA...",Cameroon
1,HIV-Free Project in the Northwest and Southwes...,EGPAF provides support to CBCHS on this CDC-fu...,Cameroon
2,Strengthening the Monitoring and Evaluation of...,"With support from UNICEF, EGPAF is providing t...",Cameroon
3,Call to Action (USAID/PEPFAR),With funding from the United States Agency for...,Cameroon
4,The Delivering Technical Assistance Project,The ECHO (Extension for Community Healthcare O...,Cameroon
...,...,...,...
148,Elimination of Mother-to-Child HIV Transmissio...,This Johnson & Johnson–funded project supports...,Zimbabwe
149,Delivering Technical Assistance Project (DELTA),EGPAF is implementing two projects under the C...,Zimbabwe
150,"Catalyzing Expanded Access to Early Testing, C...",The overarching goal of this Unitaid-funded pr...,Zimbabwe
151,ACT Adolescent HIV Project,The objectives of this two-year project funded...,Zimbabwe


In [13]:
#grab country pdfs and put them in a folder, take a screen shoot of the country page and print the url of countries without a pdf to download. 

png_files=[]

mime_types = "application/pdf,application/vnd.adobe.xfdf,application/vnd.fdf,application/vnd.adobe.xdp+xml"

fp = webdriver.FirefoxProfile()
fp.set_preference("browser.download.folderList", 2)
fp.set_preference("browser.download.manager.showWhenStarting", False)
fp.set_preference("browser.download.dir", os.getcwd())
fp.set_preference("browser.helperApps.neverAsk.saveToDisk", mime_types)
fp.set_preference("plugin.disable_full_page_plugin_for_types", mime_types)
fp.set_preference("pdfjs.disabled", True)

browser = webdriver.Firefox(firefox_profile=fp)


for country in countries_link:
    
    try:
        
        browser.get(country)
        
        file_name = country.split('/')[-2]+".png"
        browser.save_screenshot(file_name)
        png_files.append(file_name)


        elm=browser.find_element_by_xpath('//*[@id="main"]/article/div[2]/div[1]/div/div/a')
        elm.click()
        
        
        
    except:
        
        print(country)




https://www.pedaids.org/country/india/
https://www.pedaids.org/country/nigeria/
https://www.pedaids.org/country/rwanda/
